In [ ]:
import os
import sys
import glob
import numpy as np
from skimage import io
import matplotlib.pyplot as plt

try:
    del sys.modules['threshold_model']
except:
    pass
try:
    del sys.modules['eval_utils']
except:
    pass

from threshold_model import ThresholdModel
import eval_utils

# set figure properties
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

data_dir = '../../data/train/frames/rgb'

In [ ]:
# try plotting an image
filename = '%s/rgb-146.jpg' %(data_dir)
image = io.imread(filename)
height, width, channels = image.shape
print 'image has height %d, width %d, channels %d' %(height, width, channels)
plt.imshow(image)

In [ ]:
# try cropping an image
cropx_start = 200
cropx_end = 230
cropy_start = 60
cropy_end = 85

image_cropped = image[cropx_start:cropx_end, cropy_start:cropy_end, :]
plt.imshow(image_cropped)

In [ ]:
# create a model
model = ThresholdModel()
model.set_ref_mat(image_cropped)
model.set_crop(cropx_start, cropx_end, cropy_start, cropy_end)
model.set_pixel_change_threshold(5000) # this threshold determines whether the pixel "changed"
model.set_image_score_threshold(.01) # this threshold determines how many changed pixels should result in prediction=1

In [ ]:
# evaluate accuracy on a list of files
num_files = len(glob.glob('../../data/train/frames/rgb/*.jpg'))
filelist = []
for file_idx in range(num_files):
    filename = '../../data/train/frames/rgb/rgb-%d.jpg' %(file_idx)
    filelist.append(filename)

with open('../../data/train/labels.txt') as f:
    labels = f.read().strip().split('\n')
labels = [int(x) for x in labels]
labels.append(0)

for file_idx in range(num_files):
    filename = filelist[file_idx]
    filelabel = labels[file_idx]
    image_prediction, image_score = model.predict(filename)
    print '%s: label %d, prediction %d, image score %.3f' %(filename, filelabel, image_prediction, image_score)

predictions, scores, accuracy, precision, recall, fscore = eval_utils.evaluate_accuracy(filelist, labels, model)
print 'change thresh: %.3f, score thresh: %.3f, acc: %.3f, prec: %.3f, rec: %.3f, fscore: %.3f' %(pixel_change_thresh, image_score_thresh, accuracy, precision, recall, fscore)

